In [9]:
from qcg.generators import gen_supremacy, gen_hwea
from helper_fun import evaluate_circ, get_evaluator_info
from qiskit import Aer

circ = gen_hwea(3,4)
evaluator_info = get_evaluator_info(circ=circ,device_name='ibmq_boeblingen',
                                    fields=['device','basis_gates','coupling_map','properties','initial_layout','noise_model','num_shots'])
qasm_noise = evaluate_circ(circ=circ, backend='noisy_qasm_simulator', evaluator_info=evaluator_info)
print('No mitigation:', qasm_noise)

No mitigation: [0.03466796875, 0.0146484375, 0.4501953125, 0.01806640625, 0.02978515625, 0.0068359375, 0.42431640625, 0.021484375]


In [ ]:
def partial_readout_mitigation(num_shots,device,initial_layout):
    if num_shots>device.configuration().max_shots:
        print('During readout mitigation, num_shots %.3e exceeded hardware max'%num_shots)
        num_shots = device.configuration().max_shots
    filter_begin = time()
    properties = device.properties()
    num_qubits = len(properties.qubits)

    # Generate the calibration circuits
    qr = QuantumRegister(num_qubits)
    qubit_list = []
    _initial_layout = initial_layout.get_physical_bits()
    for q in _initial_layout:
        if 'ancilla' not in _initial_layout[q].register.name:
            qubit_list.append(q)
    meas_calibs, state_labels = complete_meas_cal(qubit_list=qubit_list, qr=qr, circlabel='mcal')
    print('Calculating measurement filter, %d-qubit calibration circuits * %d * %.3e shots.'%(len(meas_calibs[0].qubits),len(meas_calibs),num_shots),end=' ')
    assert len(meas_calibs)<=device.configuration().max_experiments/3*2

    # Execute the calibration circuits
    meas_calibs_transpiled = transpile(meas_calibs, backend=device)
    qobj = assemble(meas_calibs_transpiled, backend=device, shots=num_shots)
    job = device.run(qobj)
    cal_results = job.result()

    meas_fitter = CompleteMeasFitter(cal_results, state_labels, qubit_list=qubit_list, circlabel='mcal')
    meas_filter = meas_fitter.filter
    filter_time = time() - filter_begin
    print('%.3e seconds'%filter_time)
    return meas_filter

meas_filter = partial_readout_mitigation(evaluator_info['num_shots'],Aer.get_backend('qasm_simulator'),initial_layout)
evaluator_info['meas_filter'] = meas_filter
qasm_noise_mitigated = evaluate_circ(circ=circ, backend='noisy_qasm_simulator', evaluator_info=evaluator_info)
print('With mitigation:', qasm_noise_mitigated)